<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Description" data-toc-modified-id="Description-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Description</a></span></li><li><span><a href="#Load" data-toc-modified-id="Load-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load</a></span></li><li><span><a href="#Test-runs" data-toc-modified-id="Test-runs-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Test runs</a></span></li><li><span><a href="#Interact" data-toc-modified-id="Interact-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Interact</a></span></li></ul></div>

# Description

Interactive notebook to test the model and the different parameters.

# Load

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from src.cell_models import cell

import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import interact, fixed, interact_manual

from statsmodels.tsa.stattools import acf

from scipy.signal import find_peaks

In [ ]:
from src.analysis import get_cycle_stats

In [ ]:
from src.plot_utils import run_and_plot_test

# Test runs

Build something with interact so that you can get a rough sense of how the system behaves. 

In [ ]:
# cell_test, periods= run_and_plot_test()

# Interact

In [ ]:
step=.1

In [ ]:
interact_manual(
    run_and_plot_test, 
    alpha=fixed(.1),#(.1, 10., step), 
    beta0=(.01, .30, .01), 
    epsilon=fixed(.2), #(.1, 1., step), 
    gamma=(.01, .05, .01), 
    time_SG2 = fixed(12),#(1, 15), 
    transition_th=fixed(1), 
    delta=fixed(1.),
    dt=fixed(1e-1), 
    division=fixed("timer"), 
    transition=fixed("size"), # "size" or "RBc"
    k_trans=fixed(1000),
    T=fixed(10000)
);

Result for size-based transition seem to be insensitive to threshold mass, and alpha. is that true? 